# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [4]:
import anthropic
from openai import OpenAI
from IPython.display import Markdown, display, update_display
import ollama
 

In [6]:
MODEL_ANTHROPIC = "claude-3-haiku-20240307"
MODEL_GPT = "gpt-4o-mini"
MODEL_LLAMA = "llama3.2"
MAX_TOKENS = 1024

In [7]:
# Environment is set using flake.nix

In [29]:
system_prompt = """
You are an expert code reviewer specialized in python language.
You'll receive python code and explain it in simple but comprehensive terms.
You'll receive python code and review it for proposing better practices, suggest improvements in terms of readability and/or efficiency.
User markdown syntax for the output response.
"""

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [30]:
# Get anthropic to answer
_anthropic = anthropic.Client()

messages = [
    {"role": "user", "content": question}
]

response = _anthropic.messages.create(
    model=MODEL_ANTHROPIC,
    max_tokens=MAX_TOKENS,
    system=system_prompt,
    messages=messages
)

Markdown(response.content[0].text)

The code you provided is a Python generator expression that extracts the unique authors from a list of books.

Here's a breakdown of the code:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

1. `{book.get("author") for book in books if book.get("author")}`: This part is a set comprehension. It creates a set of unique author names from the `books` list. The `book.get("author")` expression retrieves the "author" key from each book dictionary, and the `if book.get("author")` condition ensures that only books with an author value are included.

2. `yield from`: This part is a generator expression syntax. The `yield from` statement is used to delegate the generation of values to another generator. In this case, it's yielding the values from the set created in the previous step, one by one.

The purpose of this code is to efficiently generate a sequence of unique author names from the `books` list. The use of a generator expression (instead of a regular list) allows the authors to be generated one at a time, which can be more memory-efficient when dealing with large datasets.

This code could be used, for example, in a function that needs to provide a list of unique authors from a collection of books.

Here's an example of how this code could be used:

```python
def get_unique_authors(books):
    return list(yield from {book.get("author") for book in books if book.get("author")})

# Example usage
books = [
    {"title": "Book 1", "author": "John Doe"},
    {"title": "Book 2", "author": "Jane Smith"},
    {"title": "Book 3", "author": "John Doe"},
    {"title": "Book 4", "author": None},
]

unique_authors = get_unique_authors(books)
print(unique_authors)  # Output: ['John Doe', 'Jane Smith']
```

In this example, the `get_unique_authors` function uses the generator expression to extract the unique authors, and the `yield from` statement to yield the values one by one. The resulting generator is then converted to a list using the `list()` function.

In [31]:
# Get gpt-4o-mini to answer, with streaming
openai = OpenAI()

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": question}
]

stream = openai.chat.completions.create(
    model=MODEL_GPT,
    messages=messages,
    stream=True
)
    
response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

The provided line of code is a generator expression that yields authors from a collection of books, specifically filtering out any books that do not have an author specified. Let's break it down step by step:

### Explanation:

1. **`{book.get("author") for book in books if book.get("author")}`**:
   - This part is a set comprehension. It's creating a set of authors extracted from a list or collection called `books`.
   - `book.get("author")` retrieves the value associated with the key `"author"` for each `book` in `books`.
   - The `if` condition ensures that only books with a defined `author` are considered. If a book does not have an author (i.e., `None` or not present), it is skipped.

2. **`yield from`**:
   - This part of the code is used to yield values from a generator. It effectively delegates the yielding of values to the set comprehension, allowing these values to be returned one by one from the function that contains this line.
   - `yield from` is especially useful for yielding values from another iterable without the need for an explicit loop.

### What the Code Does:
- This line of code yields from a set of unique authors found in the `books` collection. Any books that do not have an author will be excluded, ensuring that all yielded authors are defined.

### Use Case:
- This could be used in a function that collects and processes unique authors from a large collection of books, maybe to list them or do further processing on them.

### Suggestions for Improvement:
The code is fairly concise and achieves its purpose, but there are a couple of aspects that could enhance readability and maintainability:

1. **Use a List Comprehension if Duplicates Are Needed**: If the requirement is to consider multiple instances of the same author, then a list comprehension (or generator expression) would be more appropriate than a set comprehension. A list can include duplicates and retains the order of appearance.

2. **Add Comments**: Consider adding comments to clarify the intention of the code. This will help other developers (or your future self) better understand the purpose of the line.

3. **Function Context**: If this line is used inside a function, naming the function meaningfully and accompanying it with a docstring explaining its functionality would improve clarity.

Here's a refactored version considering readability:

python
def yield_authors(books):
    """
    Yield unique authors from the provided list of books.
    
    Args:
        books (list): A list of dictionaries containing book information.
    
    Yields:
        The unique authors from the books.
    """
    unique_authors = {book.get("author") for book in books if book.get("author")}
    yield from unique_authors


### Conclusion:
The original line of code effectively yields unique authors from the book list, but providing additional context and ensuring it fits the specific use case will improve its usefulness in a broader codebase.

In [32]:
# Get Llama 3.2 to answer, with streaming

ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": question}
]

stream = ollama.chat.completions.create(
    model=MODEL_LLAMA,
    messages=messages,
    stream=True
)

response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

**Code Explanation**

# Code Description


This line of code uses Python's generator feature to iterate over a list of book dictionaries, filtering out any authors that don't exist. It's using the `yield from` syntax, introduced in Python 3.3, which allows us to delegate iteration to another iterable.

Here's what happens step by step:

1. `{book.get("author") for book in books if book.get("author")}`: This is a generator expression that creates an iterator over the authors of each book.
2. `yield from`: This keyword tells Python to delegate the iteration to the previous part of the expression.

**Code Breakdown**
python
# Importing necessary modules (none needed for this snippet)
import pandas as pd  # Assuming 'books' is a pandas DataFrame

# Sample data
books = pd.DataFrame({
    "book_id": [1, 2, 3],
    "title": ["Book A", "Book B", "Book C"],
    "author": ["Author X", None, "Author Y"]  # Missing author for Book B
})

# Filter books with existing authors and yield them as a sequence of authors
yield from {book.get("author") for book in books if book.get("author")}


In this example, `books` is a pandas DataFrame containing information about various books. The line iterates over each row (`book`) in the DataFrame:

*   If a book has an author (i.e., `"author"` exists), it yields that author.
*   If a book is missing an author, it skips it.

To use this generator outside of its context, you would need to assign it to a variable or process the result more explicitly. Here's an updated code snippet demonstrating this:

python
# Example usage:
books_with_authors = list(yield from {book.get("author") for book in books if book.get("author")})

print(books_with_authors)  # Output: ['Author X', 'Author Y']


However, using `list(x)` on a generator can be inefficient since it first requires the entire sequence be generated before being stored.

In general, use cases like this typically involve iterating over an iterable (e.g., files, books), filtering or mapping its elements, and then processing those results individually. In such situations, working directly with iterators is often more efficient than creating intermediate sequences to store the processed data.

In [1]:
import ollama

In [2]:
messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [6]:
MODEL_DEEPSEEK = 'deepseek-r1'
stream = ollama.chat(model=MODEL_DEEPSEEK, messages=messages, stream=True)
 
response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    if 'message' in chunk:
        content = chunk['message']['content']
        response += content
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

<think>
Okay, so I need to describe some business applications of Generative AI. Hmm, I'm not exactly sure what all that entails, but I'll try to think through it.

First off, what is Generative AI? From what I remember, it's a type of AI that can generate content like text, images, music, or even video. It uses machine learning models, maybe something like GANs (Generative Adversarial Networks) or transformers. So it's not just about creating random stuff; it's based on patterns learned from data.

Alright, so what kinds of businesses could benefit from this? Well, content creation comes to mind. Like generating text for articles or marketing copy. But wait, isn't that similar to what other AI tools do? Maybe if it's more specialized, like in a specific domain, it could be better. For example, an AI tailored for legal documents might create something more accurate than generic models.

Then there's image generation. I think this is used in art or design, but also maybe in marketing with ads or social media content. It can help companies quickly generate visuals without needing human artists, which would save time and cost.

 augmenting data is another area. If a company has limited data, using AI to create synthetic datasets could be useful for training models without relying on external data sources. This might be especially helpful in fields like healthcare or finance where data privacy is a big concern.

Personalization seems important too. Tailoring content based on individual customer behavior or preferences. Maybe something like targeted ads that change based on what the user clicks on previously. But how does Generative AI handle that? It must analyze user data and use it to inform the generation process.

Marketing campaigns could benefit from this as well, perhaps creating more engaging and relevant content than traditional methods. Maybe even automating parts of campaign creation or optimizing ad copy using AI-generated variations based on performance metrics.

In creative industries like gaming or film, AI could help in generating game levels or storylines. This might speed up the process and allow for more diverse outputs since the AI can explore different possibilities beyond human creativity.

Translation services are another thought. While there are already translation tools, maybe a Generative AI could provide more accurate or contextually appropriate translations by learning from vast language corpora.

Optimizing supply chains might be possible too. By analyzing historical data, an AI could generate forecasts or suggest optimal inventory levels, reducing waste and stockouts.

Fraud detection is always a critical area for businesses. An AI that can analyze transactions in real-time to spot anomalies would be invaluable. It could detect unusual patterns more quickly than manual checks.

Accessibility features like alt text generation for images on the web are useful because they make content more accessible to visually impaired users, which also helps SEO and inclusivity overall.

Customer service could use this by generating responses that mimic human interactions based on templates and user behavior analysis. This would improve how chatbots respond to customer inquiries.

Educational tools might incorporate AI to create personalized learning materials for students, tailoring the content to their individual needs or progress over time.

I'm not sure about healthcare yet. Maybe image segmentation using AI-generated masks could help in medical imaging by identifying regions of interest more accurately than humans sometimes do.

Wait, but I should check if these are all feasible and if there aren't any major challenges. For example, ensuring the generated content is accurate or safe, especially in sensitive areas like legal documents or fraud detection. Also, data privacy concerns with generating synthetic datasets could be a hurdle that needs addressing.

I think I've covered several areas: content creation (text, images), marketing and sales, personalization, creative industries, translation, supply chain, fraud detection, accessibility, customer service, education, and healthcare imaging. Maybe there are more, but these seem like solid starting points.

Now, to structure this into a coherent answer, I should probably list each application with a brief explanation of how Generative AI is used in that context. That way, it's clear and organized.
</think>

Generative AI offers transformative applications across various industries by leveraging its ability to create high-quality content, data augmentation, personalization, and optimization. Here are some key business applications:

1. **Content Creation**: 
   - **Text Generation**: AI can craft specialized legal documents or marketing copy with precision, enhancing both accuracy and creativity.
   - **Image Generation**: Assistive tools for artists, marketers, and designers to generate visuals quickly, reducing reliance on human talent.

2. **Data Augmentation**:
   - Create synthetic datasets to augment existing data, particularly useful in privacy-sensitive sectors like healthcare and finance for model training without external data use.

3. **Personalization**:
   - Tailor content dynamically based on user behavior and preferences, enhancing customer experience through personalized ads or marketing campaigns.

4. **Marketing Campaigns**:
   - Automate and optimize content creation, offering dynamic ad copy variations to improve engagement and effectiveness.

5. **Creative Industries**:
   - Accelerate game development by generating levels and storylines, exploring diverse creative possibilities beyond human capabilities.
   - Enhance film production with AI-generated content for diverse and innovative storylines.

6. **Translation Services**:
   - Provide contextually appropriate translations using advanced language models, surpassing traditional tools in accuracy and relevance.

7. **Supply Chain Optimization**:
   - Analyze data to generate optimized forecasts and inventory levels, reducing waste and stockouts through intelligent predictions.

8. **Fraud Detection**:
   - Real-time transaction analysis by AI to identify anomalies, offering a proactive approach to detecting fraudulent activities quickly.

9. **Accessibility**:
   - Generate alt text for images, improving web accessibility, SEO, and inclusivity for visually impaired users.

10. **Customer Service**:
    - Enhance chatbots with AI-generated responses that reflect human-like interactions based on user behavior analysis, improving support efficiency.

11. **Educational Tools**:
    - Personalize learning materials for students by adapting content to their individual needs and progress, enhancing educational outcomes.

12. **Healthcare Imaging**:
    - Use AI to generate precise image masks for medical imaging, aiding in accurate diagnosis through improved segmentation techniques.

Each application leverages Generative AI's unique capabilities, addressing challenges like accuracy, safety, and data privacy while driving innovation across sectors.